In [1]:
path_for_joined_geotiff_df = '' 
path_for_1km_by_1km_folder = ''
path_to_save_tf_dataset = ''

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
#from google.colab import drive
import pandas as pd
from osgeo import gdal
from skimage.transform import resize
#drive.mount('/content/drive')
import random

Mounted at /content/drive


In [3]:
df = pd.read_csv(path_for_joined_geotiff_df)

In [5]:
#These Rows (from yoesmetie) had random points with elevation in the billions, so I just exluded them
outliers = [964, 970, 975, 976, 977, 982, 983, 984, 985, 992, 993, 994, 995]
df = df[~df.index.isin(outliers)]

In [6]:
image_size = 128
batch_size = 32
num_channels = 1
splits = 1000//image_size

In [7]:
#Gets file name from the df and reads raster data from 1km_by_1km file
file_head = path_for_1km_by_1km_folder
dataset = []
labels = []
for index, row in df.iterrows():
  path =  file_head + '/' +str(row.file_name)
  ds = gdal.Open(path)
  ds = np.array(ds.GetRasterBand(1).ReadAsArray())
  ds[ds < 0] = 0
  #Some tiles are .5 meter resolution, size 2000*2000, and some are 2 meter resolution, size 500*500.
  if np.shape(ds) != (1000, 1000):
    #resizes all tiles to 1000*1000
    ds = resize(ds, (1000, 1000))
  for i in range(splits):
    ds_split = ds[image_size*i:image_size*(i+1), image_size*i:image_size*(i+1)]
    dataset.append(ds_split)
    labels.append(row.BIOME_NAME)

In [8]:
steps_per_epoch = int(np.ceil(len(dataset)/batch_size))
num_classes = len(np.unique(labels))

In [9]:
def normalize(dataset):
  return 2*(dataset - np.min(dataset))/(np.max(dataset) - np.min(dataset))

In [10]:
dataset = normalize(dataset)

In [11]:
#Adds channel to data
dataset = np.expand_dims(dataset, axis=-1)

In [12]:
#Class Labels
labels = pd.factorize(labels)
all_labels = keras.utils.to_categorical(labels[0], num_classes)

In [13]:
#Creates tesnor of raster data with class labels
dataset = tf.data.Dataset.from_tensor_slices((dataset, all_labels))
dataset = dataset.shuffle(buffer_size=len(dataset)).batch(batch_size)

In [14]:
tf.data.experimental.save(
    dataset, path_for_folder_to_save_data, compression=None, shard_func=None, checkpoint_args=None)